Question 5 Attention Mechanism

In [1]:
import wandb

wandb.login(key="55f4fea427568f21b05f76ecad678f380953242f")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m015 (da24m015-iitm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import wandb
from tqdm import tqdm

# Dataset utilities
class TransliterationDataset(Dataset):
    def __init__(self, pairs, input_vocab, output_vocab):
        self.pairs = pairs
        self.input_vocab = input_vocab
        self.output_vocab = output_vocab
        self.sos = output_vocab['<sos>']
        self.eos = output_vocab['<eos>']

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        source, target = self.pairs[idx]
        input_ids = [self.input_vocab[c] for c in source]
        target_ids = [self.sos] + [self.output_vocab[c] for c in target] + [self.eos]
        return torch.tensor(input_ids), torch.tensor(target_ids)

def build_vocab(pairs):
    input_chars = set()
    output_chars = set()
    for source, target in pairs:
        input_chars.update(source)
        output_chars.update(target)
    input_vocab = {c: i + 1 for i, c in enumerate(sorted(input_chars))}
    input_vocab['<pad>'] = 0
    output_vocab = {c: i + 3 for i, c in enumerate(sorted(output_chars))}
    output_vocab.update({'<pad>': 0, '<sos>': 1, '<eos>': 2})
    return input_vocab, output_vocab

def load_pairs(path):
    df = pd.read_csv(path, sep="\t", header=None, names=["target", "source", "count"], dtype=str)
    df.dropna(subset=["source", "target"], inplace=True)
    return list(zip(df["source"], df["target"]))

def collate_fn(batch):
    inputs, targets = zip(*batch)
    input_lens = [len(seq) for seq in inputs]
    target_lens = [len(seq) for seq in targets]
    inputs_padded = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded, input_lens, target_lens

class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embed_size, padding_idx=0)
        rnn_class = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[cell_type]
        self.rnn = rnn_class(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs, mask):
        timestep = encoder_outputs.size(1)
        hidden = hidden[-1].unsqueeze(1).repeat(1, timestep, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy @ self.v
        energy.masked_fill_(mask == 0, -1e10)
        return torch.softmax(energy, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(output_size, embed_size, padding_idx=0)
        rnn_class = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[cell_type]
        self.rnn = rnn_class(embed_size + hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.attention = BahdanauAttention(hidden_size)

    def forward(self, input_token, hidden, encoder_outputs, mask):
        embedded = self.embedding(input_token.unsqueeze(1))
        attn_weights = self.attention(hidden[0] if isinstance(hidden, tuple) else hidden, encoder_outputs, mask)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)
        rnn_input = torch.cat([embedded, context], dim=2)
        output, hidden = self.rnn(rnn_input, hidden)
        output = self.fc(torch.cat([output.squeeze(1), context.squeeze(1)], dim=1))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def create_mask(self, src):
        return (src != 0).float()

    def forward(self, src, src_lens, tgt=None, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        device = src.device
        encoder_outputs, hidden = self.encoder(src, src_lens)
        mask = self.create_mask(src).to(device)

        if tgt is not None:
            tgt_len = tgt.size(1)
            outputs = torch.zeros(batch_size, tgt_len, self.decoder.fc.out_features, device=device)
            input_token = tgt[:, 0]
            for t in range(1, tgt_len):
                output, hidden = self.decoder(input_token, hidden, encoder_outputs, mask)
                outputs[:, t] = output
                teacher_force = torch.rand(1).item() < teacher_forcing_ratio
                input_token = tgt[:, t] if teacher_force else output.argmax(1)
            return outputs
        else:
            predictions = []
            input_token = torch.tensor([1] * batch_size, device=device)  # <sos>
            for _ in range(20):
                output, hidden = self.decoder(input_token, hidden, encoder_outputs, mask)
                top1 = output.argmax(1)
                predictions.append(top1.unsqueeze(1))
                input_token = top1
            return torch.cat(predictions, dim=1)

def accuracy(preds, targets, pad_idx=0):
    pred_tokens = preds.argmax(dim=-1)
    correct = ((pred_tokens == targets) & (targets != pad_idx)).sum().item()
    total = (targets != pad_idx).sum().item()
    return correct / total if total > 0 else 0.0

def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, total_acc = 0, 0
    for src, tgt, src_lens, tgt_lens in tqdm(loader, desc="Training", leave=False):
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, src_lens, tgt)
        loss = criterion(output[:, 1:].reshape(-1, output.size(-1)), tgt[:, 1:].reshape(-1))
        acc = accuracy(output[:, 1:], tgt[:, 1:])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_acc = 0, 0
    for src, tgt, src_lens, tgt_lens in tqdm(loader, desc="Evaluating", leave=False):
        src, tgt = src.to(device), tgt.to(device)
        output = model(src, src_lens, tgt, teacher_forcing_ratio=0.0)
        loss = criterion(output[:, 1:].reshape(-1, output.size(-1)), tgt[:, 1:].reshape(-1))
        acc = accuracy(output[:, 1:], tgt[:, 1:])
        total_loss += loss.item()
        total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

def main():
    wandb.init(project="dakshina-transliteration", config=wandb.config)
    config = wandb.config

    def generate_run_name(cfg):
        return f"attn_cell:{cfg.cell_type}_embed:{cfg.embed_size}_hid:{cfg.hidden_size}_layers:{cfg.num_layers}"

    wandb.run.name = generate_run_name(config)
    wandb.run.save()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_pairs = load_pairs("/kaggle/input/dakshina-dataset/hi/lexicons/hi.translit.sampled.train.tsv")
    dev_pairs = load_pairs("/kaggle/input/dakshina-dataset/hi/lexicons/hi.translit.sampled.dev.tsv")
    input_vocab, output_vocab = build_vocab(train_pairs)
    train_dataset = TransliterationDataset(train_pairs, input_vocab, output_vocab)
    dev_dataset = TransliterationDataset(dev_pairs, input_vocab, output_vocab)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(len(input_vocab), config.embed_size, config.hidden_size, config.num_layers, config.cell_type, config.dropout)
    decoder = Decoder(len(output_vocab), config.embed_size, config.hidden_size, config.num_layers, config.cell_type, config.dropout)
    model = Seq2Seq(encoder, decoder).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(10):
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        wandb.log({"epoch": epoch, "train_loss": train_loss, "train_accuracy": train_acc, "val_loss": val_loss, "val_accuracy": val_acc})

if __name__ == "__main__":
    sweep_config = {
        "method": "bayes",
        "metric": {"name": "val_accuracy", "goal": "maximize"},
        "parameters": {
            "embed_size": {"values": [64, 128]},
            "hidden_size": {"values": [64, 128]},
            "num_layers": {"values": [1, 2]},
            "cell_type": {"values": ["GRU", "LSTM"]},
            "dropout": {"values": [0.2, 0.3]},
            "lr": {"min": 0.0001, "max": 0.01},
            "batch_size": {"values": [32, 64]}
        }
    }
    sweep_id = wandb.sweep(sweep_config, project="dakshina-transliteration")
    wandb.agent(sweep_id, function=main, count=25)

Create sweep with ID: olaafvbq
Sweep URL: https://wandb.ai/da24m015-iitm/dakshina-transliteration/sweeps/olaafvbq


wandb: Agent Starting Run: dqsxk6ls with config:
wandb: 	batch_size: 32
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embed_size: 128
wandb: 	hidden_size: 128
wandb: 	lr: 0.009774819477897757
wandb: 	num_layers: 1


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▇██▇▇████
train_loss,█▂▁▁▁▂▁▁▁▁
val_accuracy,▃▇▁▆█▃▂▅█▅
val_loss,▇▁█▅▅▅▆▇▄▂
epoch,9
train_accuracy,0.66739
train_loss,1.09943
val_accuracy,0.58549
val_loss,1.42533


wandb: Agent Starting Run: hhe5g3qf with config:
wandb: 	batch_size: 32
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embed_size: 128
wandb: 	hidden_size: 128
wandb: 	lr: 0.007335588748686146
wandb: 	num_layers: 1


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▇█████▇█▇
train_loss,█▂▁▁▁▁▁▂▁▂
val_accuracy,▁▅▅▄▅▇▄█▄▁
val_loss,▄▁▃▅▂▅▆▂▆█
epoch,9
train_accuracy,0.69899
train_loss,0.99105
val_accuracy,0.60141
val_loss,1.3677


...

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▅▆▇▇▇████
train_loss,█▃▂▂▂▂▁▁▁▁
val_accuracy,▁▄▆▅▇█▆██▇
val_loss,█▆▂▃▂▁▃▁▂▄
epoch,9
train_accuracy,0.80676
train_loss,0.62541
val_accuracy,0.70861
val_loss,1.06765


wandb: Agent Starting Run: ychap5z3 with config:
wandb: 	batch_size: 32
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	embed_size: 64
wandb: 	hidden_size: 64
wandb: 	lr: 0.008668925844398698
wandb: 	num_layers: 2


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▆▆▇▇█████
train_loss,█▃▂▂▂▁▁▁▁▁
val_accuracy,▁▃▅▆▇██▇█▇
val_loss,█▅▂▄▁▂▁▂▁▂
epoch,9
train_accuracy,0.77849
train_loss,0.71876
val_accuracy,0.68748
val_loss,1.07398


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: a3pgr49z with config:
wandb: 	batch_size: 32
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.3
wandb: 	embed_size: 128
wandb: 	hidden_size: 128
wandb: 	lr: 0.001590516685905841
wandb: 	num_layers: 2


Training:  21%|██        | 291/1382 [00:11<00:41, 26.56it/s]  

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import csv

# ---------------- Dataset & Utils ----------------
class TransliterationDataset(Dataset):
    def __init__(self, pairs, input_vocab, output_vocab):
        self.pairs = pairs
        self.input_vocab = input_vocab
        self.output_vocab = output_vocab
        self.sos = output_vocab['<sos>']
        self.eos = output_vocab['<eos>']

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        source, target = self.pairs[idx]
        input_ids = [self.input_vocab[c] for c in source]
        target_ids = [self.sos] + [self.output_vocab[c] for c in target] + [self.eos]
        return torch.tensor(input_ids), torch.tensor(target_ids)

def load_pairs(path):
    df = pd.read_csv(path, sep='\t', header=None, names=['target', 'source', 'count'], dtype=str)
    df.dropna(subset=["source", "target"], inplace=True)
    return list(zip(df['source'], df['target']))

def build_vocab(pairs):
    input_chars = set()
    output_chars = set()
    for src, tgt in pairs:
        input_chars.update(src)
        output_chars.update(tgt)
    input_vocab = {c: i+1 for i, c in enumerate(sorted(input_chars))}
    input_vocab['<pad>'] = 0
    output_vocab = {c: i+3 for i, c in enumerate(sorted(output_chars))}
    output_vocab.update({'<pad>': 0, '<sos>': 1, '<eos>': 2})
    return input_vocab, output_vocab

def collate_fn(batch):
    inputs, targets = zip(*batch)
    input_lens = [len(x) for x in inputs]
    target_lens = [len(x) for x in targets]
    inputs_padded = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded, input_lens, target_lens

# ---------------- Bahdanau Attention ----------------
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size, attn_size):
        super().__init__()
        self.W1 = nn.Linear(hidden_size, attn_size)
        self.W2 = nn.Linear(hidden_size, attn_size)
        self.V = nn.Linear(attn_size, 1)

    def forward(self, hidden, encoder_outputs, mask=None):
        # hidden: (num_layers * num_directions, batch, hidden_size) or (batch, hidden_size)
        # encoder_outputs: (batch, seq_len, hidden_size)
        # We'll take hidden from last layer (batch, hidden_size)
        if hidden.dim() == 3:
            hidden = hidden[-1]  # take last layer, shape: (batch, hidden_size)
        hidden_with_time_axis = hidden.unsqueeze(1)  # (batch, 1, hidden_size)
        score = self.V(torch.tanh(self.W1(encoder_outputs) + self.W2(hidden_with_time_axis)))  # (batch, seq_len, 1)
        attn_weights = torch.softmax(score, dim=1)  # (batch, seq_len, 1)
        if mask is not None:
            attn_weights = attn_weights * mask.unsqueeze(2)  # apply mask
            attn_weights = attn_weights / (attn_weights.sum(dim=1, keepdim=True) + 1e-10)
        context_vector = torch.sum(attn_weights * encoder_outputs, dim=1)  # (batch, hidden_size)
        return context_vector, attn_weights.squeeze(-1)  # attn_weights shape (batch, seq_len)

# ---------------- Models ----------------
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embed_size, padding_idx=0)
        rnn_cls = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[cell_type]
        self.rnn = rnn_cls(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0, bidirectional=False)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        packed_outputs, hidden = self.rnn(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)  # (batch, seq_len, hidden_size)
        return outputs, hidden  # outputs for attention, hidden for decoder init

class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers, cell_type, dropout, attn_size):
        super().__init__()
        self.embedding = nn.Embedding(output_size, embed_size, padding_idx=0)
        rnn_cls = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[cell_type]
        self.rnn = rnn_cls(embed_size + hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.attention = BahdanauAttention(hidden_size, attn_size)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # concat context vector + rnn output

    def forward(self, input_token, hidden, encoder_outputs, mask=None):
        # input_token: (batch,), hidden: (num_layers, batch, hidden_size)
        embedded = self.embedding(input_token).unsqueeze(1)  # (batch, 1, embed_size)
        context_vector, attn_weights = self.attention(hidden, encoder_outputs, mask)  # (batch, hidden_size), (batch, seq_len)
        rnn_input = torch.cat((embedded, context_vector.unsqueeze(1)), dim=-1)  # (batch, 1, embed+hidden)
        output, hidden = self.rnn(rnn_input, hidden)  # output: (batch,1,hidden_size)
        output = output.squeeze(1)  # (batch, hidden_size)
        output = torch.cat((output, context_vector), dim=1)  # (batch, hidden_size*2)
        output = self.fc(output)  # (batch, output_size)
        return output, hidden, attn_weights

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, output_vocab):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.output_vocab = output_vocab
        self.sos = output_vocab['<sos>']
        self.eos = output_vocab['<eos>']

    def create_mask(self, src, src_lens):
        # mask for padding tokens in encoder outputs (batch, seq_len)
        batch_size, seq_len = src.size()
        mask = torch.arange(seq_len).expand(batch_size, seq_len).to(src.device) < torch.tensor(src_lens).unsqueeze(1).to(src.device)
        return mask

    def forward(self, src, src_lens, tgt=None, teacher_forcing_ratio=0.5, max_len=20):
        batch_size = src.size(0)
        encoder_outputs, hidden = self.encoder(src, src_lens)  # encoder_outputs (batch, seq_len, hidden), hidden (num_layers, batch, hidden)
        mask = self.create_mask(src, src_lens)

        # Initialize decoder input and outputs
        if tgt is not None:
            tgt_len = tgt.size(1)
        else:
            tgt_len = max_len

        outputs = torch.zeros(batch_size, tgt_len, self.decoder.fc.out_features).to(src.device)
        input_token = torch.tensor([self.sos] * batch_size).to(src.device)

        # For LSTM hidden is tuple (h,c), for others just tensor
        decoder_hidden = hidden
        if isinstance(hidden, tuple):
            decoder_hidden = (hidden[0], hidden[1])  # just keep as is

        for t in range(tgt_len):
            output, decoder_hidden, attn_weights = self.decoder(input_token, decoder_hidden, encoder_outputs, mask)
            outputs[:, t] = output
            teacher_force = tgt is not None and (torch.rand(1).item() < teacher_forcing_ratio)
            if teacher_force and t + 1 < tgt_len:
                input_token = tgt[:, t+1]
            else:
                input_token = output.argmax(1)
        return outputs

# ---------------- Train + Eval ----------------
def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for src, tgt, src_lens, _ in dataloader:
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, src_lens, tgt)
        loss = criterion(output[:, :-1].reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_and_save(model, dataloader, input_vocab, output_vocab, device, csv_path=None):
    model.eval()
    inv_input_vocab = {v: k for k, v in input_vocab.items()}
    inv_output_vocab = {v: k for k, v in output_vocab.items()}
    correct = 0
    total = 0
    results = []

    with torch.no_grad():
        for src, tgt, src_lens, _ in dataloader:
            src = src.to(device)
            batch_size = src.size(0)
            encoder_outputs, hidden = model.encoder(src, src_lens)
            mask = model.create_mask(src, src_lens)
            input_token = torch.tensor([output_vocab['<sos>']] * batch_size).to(device)
            decoder_hidden = hidden
            decoded_tokens = []

            max_len = 20
            for _ in range(max_len):
                output, decoder_hidden, attn_weights = model.decoder(input_token, decoder_hidden, encoder_outputs, mask)
                input_token = output.argmax(1)
                decoded_tokens.append(input_token.unsqueeze(1))
            decoded = torch.cat(decoded_tokens, dim=1)  # (batch, max_len)

            for i in range(batch_size):
                pred = ''.join([inv_output_vocab[t.item()] for t in decoded[i] if t.item() not in [output_vocab['<eos>'], 0]])
                truth = ''.join([inv_output_vocab[t.item()] for t in tgt[i][1:-1]])
                inp = ''.join([inv_input_vocab[t.item()] for t in src[i] if t.item() != 0])
                results.append((inp, pred, truth))
                if pred == truth:
                    correct += 1
                total += 1

    acc = correct / total * 100 if total > 0 else 0
    print(f"\nTest Accuracy: {acc:.2f}%")
    for inp, pred, truth in results[:10]:
        print(f"{inp:<15} | Pred: {pred:<20} | Truth: {truth}")

    if csv_path is not None:
        with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Input', 'Prediction', 'GroundTruth'])
            writer.writerows(results)
        print(f"\nPredictions saved to: {csv_path}")

    return acc, results

if __name__ == "__main__":
    config = {
        "embed_size": 128,
        "hidden_size": 128,
        "attn_size": 64,
        "num_layers": 2,
        "cell_type": "LSTM",
        "dropout": 0.3,
        "batch_size": 32,
        "lr": 0.001590516685905841,
        "epochs": 10,
    }

    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_pairs = load_pairs("/kaggle/input/dakshina-dataset/hi/lexicons/hi.translit.sampled.train.tsv")
    test_pairs = load_pairs("/kaggle/input/dakshina-dataset/hi/lexicons/hi.translit.sampled.test.tsv")
    input_vocab, output_vocab = build_vocab(train_pairs)
    train_dataset = TransliterationDataset(train_pairs, input_vocab, output_vocab)
    test_dataset = TransliterationDataset(test_pairs, input_vocab, output_vocab)

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(len(input_vocab), config["embed_size"], config["hidden_size"],
                      config["num_layers"], config["cell_type"], config["dropout"])
    decoder = Decoder(len(output_vocab), config["embed_size"], config["hidden_size"],
                      config["num_layers"], config["cell_type"], config["dropout"], config["attn_size"])
    model = Seq2Seq(encoder, decoder, output_vocab).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    best_acc = 0
    for epoch in range(config["epochs"]):
        train_loss = train_model(model, train_loader, optimizer, criterion, device)
        print(f"Epoch {epoch+1} Train Loss: {train_loss:.4f}")
        acc, results = evaluate_and_save(model, test_loader, input_vocab, output_vocab, device, csv_path=None)
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_model.pth")

    print("\nLoading best model for final evaluation...")
    model.load_state_dict(torch.load("best_model.pth"))
    evaluate_and_save(model, test_loader, input_vocab, output_vocab, device, csv_path="test_predictions.csv")


Epoch 1 Train Loss: 1.3512

Test Accuracy: 25.88%
ank             | Pred: अंक                  | Truth: अंक
anka            | Pred: अंका                 | Truth: अंक
ankit           | Pred: अंकित                | Truth: अंकित
anakon          | Pred: अनकों                | Truth: अंकों
ankhon          | Pred: अंखों                | Truth: अंकों
ankon           | Pred: अंकों                | Truth: अंकों
angkor          | Pred: अंगकोर               | Truth: अंकोर
ankor           | Pred: अंकोर                | Truth: अंकोर
angaarak        | Pred: अंगारक               | Truth: अंगारक
angarak         | Pred: अंगराक               | Truth: अंगारक
Epoch 2 Train Loss: 0.7973

Test Accuracy: 29.30%
ank             | Pred: अंक                  | Truth: अंक
anka            | Pred: अंका                 | Truth: अंक
ankit           | Pred: अंकित                | Truth: अंकित
anakon          | Pred: अनकों                | Truth: अंकों
ankhon          | Pred: अंखों                | Truth: अंकों
ankon 

adding heatmap

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math

# ---------------- Dataset & Utils ----------------
class TransliterationDataset(Dataset):
    def __init__(self, pairs, input_vocab, output_vocab):
        self.pairs = pairs
        self.input_vocab = input_vocab
        self.output_vocab = output_vocab
        self.sos = output_vocab['<sos>']
        self.eos = output_vocab['<eos>']

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        source, target = self.pairs[idx]
        input_ids = [self.input_vocab[c] for c in source]
        target_ids = [self.sos] + [self.output_vocab[c] for c in target] + [self.eos]
        return torch.tensor(input_ids), torch.tensor(target_ids)

def load_pairs(path):
    df = pd.read_csv(path, sep='\t', header=None, names=['target', 'source', 'count'], dtype=str)
    df.dropna(subset=["source", "target"], inplace=True)
    return list(zip(df['source'], df['target']))

def build_vocab(pairs):
    input_chars = set()
    output_chars = set()
    for src, tgt in pairs:
        input_chars.update(src)
        output_chars.update(tgt)
    input_vocab = {c: i+1 for i, c in enumerate(sorted(input_chars))}
    input_vocab['<pad>'] = 0
    output_vocab = {c: i+3 for i, c in enumerate(sorted(output_chars))}
    output_vocab.update({'<pad>': 0, '<sos>': 1, '<eos>': 2})
    return input_vocab, output_vocab

def collate_fn(batch):
    inputs, targets = zip(*batch)
    input_lens = [len(x) for x in inputs]
    target_lens = [len(x) for x in targets]
    inputs_padded = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded, input_lens, target_lens

# ---------------- Bahdanau Attention ----------------
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size, attn_size):
        super().__init__()
        self.W1 = nn.Linear(hidden_size, attn_size)
        self.W2 = nn.Linear(hidden_size, attn_size)
        self.V = nn.Linear(attn_size, 1)

    def forward(self, hidden, encoder_outputs, mask=None):
        # hidden: (num_layers * num_directions, batch, hidden_size) or (batch, hidden_size)
        # encoder_outputs: (batch, seq_len, hidden_size)
        # We'll take hidden from last layer (batch, hidden_size)
        if hidden.dim() == 3:
            hidden = hidden[-1]  # take last layer, shape: (batch, hidden_size)
        hidden_with_time_axis = hidden.unsqueeze(1)  # (batch, 1, hidden_size)
        score = self.V(torch.tanh(self.W1(encoder_outputs) + self.W2(hidden_with_time_axis)))  # (batch, seq_len, 1)
        attn_weights = torch.softmax(score, dim=1)  # (batch, seq_len, 1)
        if mask is not None:
            attn_weights = attn_weights * mask.unsqueeze(2)  # apply mask
            attn_weights = attn_weights / (attn_weights.sum(dim=1, keepdim=True) + 1e-10)
        context_vector = torch.sum(attn_weights * encoder_outputs, dim=1)  # (batch, hidden_size)
        return context_vector, attn_weights.squeeze(-1)  # attn_weights shape (batch, seq_len)

# ---------------- Models ----------------
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embed_size, padding_idx=0)
        rnn_cls = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[cell_type]
        self.rnn = rnn_cls(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0, bidirectional=False)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        packed_outputs, hidden = self.rnn(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)  # (batch, seq_len, hidden_size)
        return outputs, hidden  # outputs for attention, hidden for decoder init

class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers, cell_type, dropout, attn_size):
        super().__init__()
        self.embedding = nn.Embedding(output_size, embed_size, padding_idx=0)
        rnn_cls = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[cell_type]
        self.rnn = rnn_cls(embed_size + hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.attention = BahdanauAttention(hidden_size, attn_size)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # concat context vector + rnn output

    def forward(self, input_token, hidden, encoder_outputs, mask=None):
        # input_token: (batch,), hidden: (num_layers, batch, hidden_size)
        embedded = self.embedding(input_token).unsqueeze(1)  # (batch, 1, embed_size)
        context_vector, attn_weights = self.attention(hidden, encoder_outputs, mask)  # (batch, hidden_size), (batch, seq_len)
        rnn_input = torch.cat((embedded, context_vector.unsqueeze(1)), dim=-1)  # (batch, 1, embed+hidden)
        output, hidden = self.rnn(rnn_input, hidden)  # output: (batch,1,hidden_size)
        output = output.squeeze(1)  # (batch, hidden_size)
        output = torch.cat((output, context_vector), dim=1)  # (batch, hidden_size*2)
        output = self.fc(output)  # (batch, output_size)
        return output, hidden, attn_weights

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, output_vocab):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.output_vocab = output_vocab
        self.sos = output_vocab['<sos>']
        self.eos = output_vocab['<eos>']

    def create_mask(self, src, src_lens):
        # mask for padding tokens in encoder outputs (batch, seq_len)
        batch_size, seq_len = src.size()
        mask = torch.arange(seq_len).expand(batch_size, seq_len).to(src.device) < torch.tensor(src_lens).unsqueeze(1).to(src.device)
        return mask

    def forward(self, src, src_lens, tgt=None, teacher_forcing_ratio=0.5, max_len=20):
        batch_size = src.size(0)
        encoder_outputs, hidden = self.encoder(src, src_lens)  # encoder_outputs (batch, seq_len, hidden), hidden (num_layers, batch, hidden)
        mask = self.create_mask(src, src_lens)

        # Initialize decoder input and outputs
        if tgt is not None:
            tgt_len = tgt.size(1)
        else:
            tgt_len = max_len

        outputs = torch.zeros(batch_size, tgt_len, self.decoder.fc.out_features).to(src.device)
        input_token = torch.tensor([self.sos] * batch_size).to(src.device)

        # For LSTM hidden is tuple (h,c), for others just tensor
        decoder_hidden = hidden
        if isinstance(hidden, tuple):
            decoder_hidden = (hidden[0], hidden[1])  # just keep as is

        for t in range(tgt_len):
            output, decoder_hidden, attn_weights = self.decoder(input_token, decoder_hidden, encoder_outputs, mask)
            outputs[:, t] = output
            teacher_force = tgt is not None and (torch.rand(1).item() < teacher_forcing_ratio)
            if teacher_force and t + 1 < tgt_len:
                input_token = tgt[:, t+1]
            else:
                input_token = output.argmax(1)
        return outputs

# ---------------- New Attention Heatmap Visualization ----------------
def generate_attention_heatmap(model, test_loader, input_vocab, output_vocab, device, num_examples=9):
    model.eval()
    inv_input_vocab = {v: k for k, v in input_vocab.items()}
    inv_output_vocab = {v: k for k, v in output_vocab.items()}
    
    examples = []
    
    with torch.no_grad():
        for src, tgt, src_lens, _ in test_loader:
            if len(examples) >= num_examples:
                break
                
            src = src.to(device)
            batch_size = src.size(0)
            encoder_outputs, hidden = model.encoder(src, src_lens)
            mask = model.create_mask(src, src_lens)
            
            # Get input text
            input_text = ''.join([inv_input_vocab[t.item()] for t in src[0] if t.item() != 0])
            
            # Get target text
            target_text = ''.join([inv_output_vocab[t.item()] for t in tgt[0][1:-1]])
            
            # Initialize for decoding
            input_token = torch.tensor([output_vocab['<sos>']]).to(device)
            decoder_hidden = hidden
            
            # Store attention weights and predicted tokens
            attention_weights = []
            predicted_text = []
            
            # Decode
            max_len = 20
            for _ in range(max_len):
                output, decoder_hidden, attn_weights = model.decoder(input_token, decoder_hidden, encoder_outputs, mask)
                attention_weights.append(attn_weights[0].cpu().numpy())
                
                input_token = output.argmax(1)
                pred_token = inv_output_vocab.get(input_token.item(), '<unk>')
                
                if pred_token == '<eos>' or len(predicted_text) >= max_len:
                    break
                    
                if pred_token not in ['<sos>', '<pad>']:
                    predicted_text.append(pred_token)
            
            predicted_text = ''.join(predicted_text)
            attention_matrix = np.array(attention_weights)
            
            # Add to examples
            examples.append({
                'input': input_text,
                'target': target_text,
                'prediction': predicted_text,
                'attention': attention_matrix[:len(predicted_text), :len(input_text)]
            })
    
    # Plot attention heatmaps in a grid
    if examples:
        num_examples = min(9, len(examples))
        rows, cols = 3, 3  # Fixed 3x3 grid
        
        fig = plt.figure(figsize=(18, 18))
        
        for i, example in enumerate(examples[:num_examples]):
            if i >= rows * cols:
                break
                
            # Create subplot with more space for title
            ax = fig.add_subplot(rows, cols, i + 1)
            
            # Add title with input, target and prediction
            title = f"Input: {example['input']}\nTarget: {example['target']}\nPred: {example['prediction']}"
            ax.set_title(title, fontsize=12, pad=10)
            
            # Generate attention heatmap
            attention_data = example['attention']
            
            # Create heatmap with improved appearance
            sns.heatmap(
                attention_data, 
                ax=ax,
                xticklabels=list(example['input']),
                yticklabels=['' for _ in range(len(example['prediction']))],  # Empty y-labels like in your example
                cmap='viridis',  # Similar to your example
                vmin=0.0,        # Minimum value
                vmax=1.0,        # Maximum value
                square=True,     # Square cells
                cbar_kws={'shrink': 0.8}  # Colorbar settings
            )
            
            # Enhance the appearance of the axis labels
            plt.setp(ax.get_xticklabels(), fontsize=12, rotation=0)
            plt.setp(ax.get_yticklabels(), fontsize=12, rotation=0)
            
            # Add colorbar ticks
            colorbar = ax.collections[0].colorbar
            colorbar.set_ticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
            colorbar.set_ticklabels(['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8'])
                
        plt.tight_layout(pad=3.0)  # Add padding between subplots
        plt.savefig("attention_heatmaps.png", dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Generated attention heatmaps for {num_examples} examples")
        return examples
    else:
        print("No examples found")
        return []

# ---------------- Run ----------------
if __name__ == "__main__":
    config = {
        "embed_size": 128,
        "hidden_size": 128,
        "attn_size": 64,
        "num_layers": 2,
        "cell_type": "LSTM",
        "dropout": 0.3,
        "batch_size": 32,
        "lr": 0.001590516685905841,
        "epochs": 10,
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_pairs = load_pairs("/kaggle/input/dakshina-dataset/hi/lexicons/hi.translit.sampled.train.tsv")
    test_pairs = load_pairs("/kaggle/input/dakshina-dataset/hi/lexicons/hi.translit.sampled.test.tsv")
    input_vocab, output_vocab = build_vocab(train_pairs)
    train_dataset = TransliterationDataset(train_pairs, input_vocab, output_vocab)
    test_dataset = TransliterationDataset(test_pairs, input_vocab, output_vocab)

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(len(input_vocab), config["embed_size"], config["hidden_size"],
                      config["num_layers"], config["cell_type"], config["dropout"])
    decoder = Decoder(len(output_vocab), config["embed_size"], config["hidden_size"],
                      config["num_layers"], config["cell_type"], config["dropout"], config["attn_size"])
    model = Seq2Seq(encoder, decoder, output_vocab).to(device)

    # Load the best model
    model.load_state_dict(torch.load("best_model.pth"))
    
    # Generate attention heatmaps
    print("Generating attention heatmaps...")
    examples = generate_attention_heatmap(model, test_loader, input_vocab, output_vocab, device)

Generating attention heatmaps...


/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:80: UserWarning: Glyph 2309 (\N{DEVANAGARI LETTER A}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:80: UserWarning: Matplotlib currently does not support Devanagari natively.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:80: UserWarning: Glyph 2306 (\N{DEVANAGARI SIGN ANUSVARA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:80: UserWarning: Glyph 2325 (\N{DEVANAGARI LETTER KA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:80: UserWarning: Glyph 2309 (\N{DEVANAGARI LETTER A}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:80: UserWarning: Matplotlib currently does not support Devanagari natively.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py

Generated attention heatmaps for 9 examples
